# Imports

In [31]:
import tensorflow as tf
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras import Sequential
from tensorflow.keras.utils import plot_model
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.utils import image_dataset_from_directory

import os
import imghdr

# Limiting GPU usage for TensorFlow

In [34]:
gpus = tf.config.list_physical_devices("GPU")
for i in gpus:
    tf.config.experimental.set_memory_growth(i, True)

# Getting rid of invalid images 

In [35]:
img_folder = "images"
img_extensions = ["jpeg", "jpg", "png", "bmp"]